Simplest autoregresive moving average model can be described as:
$$
r_t = c + \phi_1 r_{t-1} + \theta_1 \epsilon{t-1} + \epsilon_t
$$
Where:
1. $r_t$ - value in period t
1. $\epsilon_t$ - error for some period
1. $c$ - baseline constant factor
1. $\phi_1$ - what part of the value last period is relevant in explaining the current one
1. $\theta_1$ - what part of the error last period is relevant in explaining the current value


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import  ARMA
from scipy.stats.distributions import chi2
sns.set()
import warnings
warnings.filterwarnings("ignore")

def LLR_test2(L1, L2, DF=1):
    LR = 2 * (L2-L1)
    return chi2.sf(LR,DF).round(3)  

raw_csv_data = pd.read_csv("Index2018.csv")
df_comp = raw_csv_data.copy()
df_comp.date = pd.to_datetime(df_comp.date, dayfirst=True)
df_comp.set_index("date", inplace=True)
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method='ffill')
df_comp['market_value']=df_comp.ftse
df_comp = df_comp.drop(columns=['spx', 'dax', 'ftse', 'nikkei'])
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]
df['returns'] = df.market_value.pct_change(1).mul(100)
df.head() #1st value NaN for returns
df = df.iloc[1:] #so we get rid of it

In [5]:
model_1_1=ARMA(df.returns,order=(1,1))
results_1_1=model_1_1.fit()
model_1_0=ARMA(df.returns,order=(1,0))
results_1_0=model_1_0.fit()
model_0_1=ARMA(df.returns,order=(0,1))
results_0_1=model_0_1.fit()
LLR_test2(results_1_0.llf, results_0_1.llf), LLR_test2(results_0_1.llf, results_1_1.llf), LLR_test2(results_1_0.llf, results_1_1.llf)

(0.579, 0.0, 0.0)

In [6]:
model_8_8=ARMA(df.returns,order=(8,8))
results_8_8=model_8_8.fit()

ValueError: The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.